In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [ ]:
def load_model():
    tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R2-Distill-Llama-8B")
    model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R2-Distill-Llama-8B",
                                                 torch_dtype=torch.bfloat16,
                                                 )
    return tokenizer, model

tokenizer, model = load_model()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (n

In [23]:
def generate_text(prompt, max_length=100):
    # Encode the prompt         
    inputs = tokenizer(prompt, return_tensors="pt")
    
    # Generate text
    with torch.no_grad():
        output = model.generate(
            inputs.input_ids,
            max_length=max_length,
            num_return_sequences=1,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            return_dict_in_generate=True,  # Add this to get a dict with sequences
            output_scores=True
        )
    
    # Get the generated token IDs
    generated_ids = output.sequences[0]
    
    # Decode the generated text with special tokens visible
    generated_text = tokenizer.decode(generated_ids, skip_special_tokens=False)

    
    return generated_text

# Example usage
prompt = '''Design a thought experiment that explores the implications of quantum entanglement on our understanding of causality and free will.
<think>
Okay, so the user is asking who's better between Sabrina Carpenter and Taylor Swift. Hmm, where do I start? First, I need to understand what they mean by "better." Better in terms of what? Music style, popularity, awards, influence? Because "better" is pretty'''
result = generate_text(prompt, max_length=2048)
print(result)


KeyboardInterrupt: 